In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
# 하이퍼 파라미터 튜닝 
# 모델의 학습 결과를 좀 더 올리기 위해서 스코어를 올리기 위해 파라미터를 바꿔보는 것

iris = load_iris()

# 10개 중에서 2개는 테스트로 쓸거고
x_train_valid, x_test, y_train_valid, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=1)

In [3]:
# 나머지 남은 8개 중에서 0.25개를 쓸 것
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.25, random_state=1)

In [5]:
print(f"train: {x_train.shape[0]}")
print(f"valid: {x_valid.shape[0]}")
print(f"test: {x_test.shape[0]}")

# test 셋은 모델이 잘 만들어져 있는지 마지막에 딱 한 번만 씀
# vaild 셋은 학습할 때 잘 되어있는지 반복적으로 씀

train: 90
valid: 30
test: 30


In [11]:
best_score = 0

# C(cost) : 이상치 허용 (낮으면 많이 허용, 높으면 적게 허용)
# gamma : 경계의 복잡도 (서포트 벡터의 영향력이 미치는 거리 - 표준 편차)

# gamma=gamma, C=C : 모델의 파라미터 부분을 바꾸는 것이 영향을 많이 미침 (이걸 찾는 과정이 하이퍼 파라미터 튜닝)

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        model = SVC(gamma=gamma, C=C)
        model.fit(x_train, y_train)
        score = model.score(x_valid, y_valid)

        # 다른 건 못 건드리지만, 감마랑 C를 바꿔가면서 score가 가장 큰 걸 찾으려고 함
        print(f"gamma:{gamma}/ C:{C} -> score:{score}")

        if score > best_score:
            best_score = score
            best_parameters = {"gamma": gamma, "C":C}

gamma:0.001/ C:0.001 -> score:0.36666666666666664
gamma:0.001/ C:0.01 -> score:0.36666666666666664
gamma:0.001/ C:0.1 -> score:0.36666666666666664
gamma:0.001/ C:1 -> score:0.6666666666666666
gamma:0.001/ C:10 -> score:0.9666666666666667
gamma:0.001/ C:100 -> score:0.9666666666666667
gamma:0.01/ C:0.001 -> score:0.36666666666666664
gamma:0.01/ C:0.01 -> score:0.36666666666666664
gamma:0.01/ C:0.1 -> score:0.6666666666666666
gamma:0.01/ C:1 -> score:0.9666666666666667
gamma:0.01/ C:10 -> score:0.9666666666666667
gamma:0.01/ C:100 -> score:0.9666666666666667
gamma:0.1/ C:0.001 -> score:0.36666666666666664
gamma:0.1/ C:0.01 -> score:0.36666666666666664
gamma:0.1/ C:0.1 -> score:0.9333333333333333
gamma:0.1/ C:1 -> score:0.9666666666666667
gamma:0.1/ C:10 -> score:0.9666666666666667
gamma:0.1/ C:100 -> score:0.9666666666666667
gamma:1/ C:0.001 -> score:0.36666666666666664
gamma:1/ C:0.01 -> score:0.36666666666666664
gamma:1/ C:0.1 -> score:0.9666666666666667
gamma:1/ C:1 -> score:0.9666666

In [8]:
# {"gamma": gamma, "C":C} 딕셔너리 형태니까 **가 있어야 함
best_model = SVC(**best_parameters)

In [9]:
best_model.fit(x_train_valid, y_train_valid)
test_score = best_model.score(x_test, y_test)

In [10]:
print(f"valid best score : {best_score}")
print(f"best params : {best_parameters}")
print(f"test score : {test_score}")

valid best score : 0.9666666666666667
best params : {'gamma': 0.001, 'C': 10}
test score : 0.9333333333333333


In [12]:
# 매번 위 과정으로 하기 어려우니까
# 그리드 서치(Grid Search)
# 가능한 모든 조합을 대상으로 하이퍼파라미터 공간을 탐색하여 최적의 조합을 찾는 방법
# 조합들을 모두 탐색하여 모델을 학습하고 검증 데이터를 사용하여 성능을 평가합니다. 
# 그리고 가장 우수한 조합과 해당 조합에서의 성능을 반환

from sklearn.model_selection import GridSearchCV

In [13]:
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, train_size=0.2, random_state=1)

In [14]:
model = SVC()
model.get_params().keys()

dict_keys(['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

In [15]:
kernel_list = ["linear", "poly", "rbf", "sigmoid"]
C_list = [1, 2]
params = {"kernel" : kernel_list, "C" : C_list}

gscv_model = GridSearchCV(model, params)
gscv_model.fit(x_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 2],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [17]:
print(f"score: {gscv_model.score(x_test, y_test)}")
print(f"best params : {gscv_model.best_params_}")
print(f"best valid score : {gscv_model.best_score_}")
print(f"best model : {gscv_model.best_estimator_}")

score: 0.95
best params : {'C': 1, 'kernel': 'linear'}
best valid score : 1.0
best model : SVC(C=1, kernel='linear')
